In [65]:
import pandas as pd
import matplotlib as plt

#Define filepaths
path_coe = 'data/COEBiddingResultsPrices.csv'
path_deregistered = 'data/MotorVehiclesDeRegisteredUnderVehicleQuotaSystemMonthly.csv'
path_registered = 'data/NewRegistrationOfMotorVehiclesUnderVehicleQuotaSystemMonthly.csv'
path_cpi = 'data/outputFile.xlsx'
path_nom_household = 'data/NominalChangeinAverageMonthlyHouseholdEmploymentIncomePerHouseholdMemberExcludingEmployerCPFContributionsAmongResidentEmployedHouseholdsbyDecilesHouseholdEmploymentIncomeAnnual20002024.csv'
#path_real_household = 'data/RealChangeinMonthlyHouseholdEmploymentIncomePerHouseholdMemberExcludingEmployerCPFContributionsAmongResidentEmployedHouseholdsatSelectedPercentilesHouseholdEmploymentIncomeAnnual20002024.csv'

#Load datasets
coe = pd.read_csv(path_coe, thousands=',') #thousands separator causing issues
deregistered = pd.read_csv(path_deregistered)
registered = pd.read_csv(path_registered)
cpi = pd.read_excel(path_cpi, sheet_name='T7', skiprows=10) #the only xlsx file
nom_household = pd.read_csv(path_nom_household)

#Function to rebase cpi
def rebase_index(df, year_base, year_start, year_end_excl):
    list_years = [i for i in range(year_start,year_end_excl)]
    list_months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    for month in list_months:
        try:
            base_year_month_value = float(df.loc[df['Period'] == f'{year_base} {month}','All Items'].iloc[0])
            for year in list_years:
                current_value = float(df.loc[df['Period'] == f'{year} {month}','All Items'].iloc[0])
                rebased_value = round((current_value / base_year_month_value) * 100,3)
                df.loc[df['Period'] == f'{year} {month}', 'All Items'] = rebased_value
        except: #dates may not be found
            print(f'Error for {year} {month}')
            pass
    return df

#Function to convert date format to YYYY-MM in digits
def convert_date_format(x):
    original_months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    converted_months = ['01','02','03','04','05','06','07','08','09','10','11','12']
    ref_dict = dict(zip(original_months, converted_months))
    this_date = x[5:]
    return f'{x[:4]}-{ref_dict[this_date]}'



In [ ]:
##### NOMINAL COE DATA #####
#cut off at last date of Nov 2025
index_of_2025Nov = coe[coe['month'] == '2025-11'].index[-1] #get the last occurrence
coe_nov25 = coe.loc[:index_of_2025Nov]
#filter down to cars A, cars B, and bikes
coe_nov25_filtered = coe_nov25[coe_nov25['vehicle_class'].isin(['Category A','Category B', 'Category D'])]
#coe_nov25_filtered[['quota', 'bids_success', 'bids_received', 'premium']] = coe_nov25_filtered[['quota', 'bids_success', 'bids_received', 'premium']].astype('int64') #in case need to convert to int
coe_nov25_filtered_bidavg = coe_nov25_filtered.groupby(['month','vehicle_class']).mean().reset_index().drop(columns=['bidding_no']) #average bids 1 and 2
coe_nov25_filtered_bidavg

,month,vehicle_class,quota,bids_success,bids_received,premium
0,2010-01,Category A,1151.5,1147.0,1507.5,19501.5
1,2010-01,Category B,702.0,698.0,994.0,20795.0
2,2010-01,Category D,375.5,371.5,530.0,870.5
3,2010-02,Category A,1151.0,1150.5,1409.5,20164.5
4,2010-02,Category B,689.0,674.0,943.0,23534.5
...,...,...,...,...,...,...
559,2025-10,Category B,819.5,814.0,1089.5,136444.5
560,2025-10,Category D,533.0,528.0,614.0,9599.5
561,2025-11,Category A,1295.5,1291.0,1634.5,109501.0
562,2025-11,Category B,803.0,780.0,1233.5,122445.5


In [ ]:
coe_nov25_filtered_bidavg[['month','vehicle_class','premium']]#.plot()

,month,vehicle_class,premium
0,2010-01,Category A,19501.5
1,2010-01,Category B,20795.0
2,2010-01,Category D,870.5
3,2010-02,Category A,20164.5
4,2010-02,Category B,23534.5
...,...,...,...
559,2025-10,Category B,136444.5
560,2025-10,Category D,9599.5
561,2025-11,Category A,109501.0
562,2025-11,Category B,122445.5


In [ ]:
cpi_t = cpi.transpose()
cpi_t.columns = cpi_t.iloc[0]
cpi_t = cpi_t[1:]
cpi_t.reset_index(inplace=True)
cpi_t.rename(columns={'index': 'Period'}, inplace=True)
cpi_mainmetric = cpi_t[['Period','All Items']] #only interested in 'All Items'
cpi_mainmetric_sorted = cpi_mainmetric[::-1] #reverse the order to have earliest month first
#cut off at earliest date of coe dataset
index_of_earliestdate = cpi_mainmetric_sorted[cpi_mainmetric_sorted['Period'] == '2010 Jan'].index[0]
cpi_mainmetric_sorted_earliestdate = cpi_mainmetric_sorted.loc[index_of_earliestdate:]
#re-base to 2010 jan = 100
cpi_mainmetric_sorted_earliestdate_rebased = rebase_index(cpi_mainmetric_sorted_earliestdate, 2010, 2010, 2026)
#convert date format to YYYY-MM
cpi_mainmetric_sorted_earliestdate_rebased.loc[:,'Period'] = cpi_mainmetric_sorted_earliestdate_rebased.loc[:,'Period'].apply(convert_date_format)
cpi_mainmetric_sorted_earliestdate_rebased

Error for 2025 Dec


Data Series,Period,All Items
190,2010-01,100.0
189,2010-02,100.0
188,2010-03,100.0
187,2010-04,100.0
186,2010-05,100.0
...,...,...
4,2025-07,133.806
3,2025-08,133.243
2,2025-09,133.591
1,2025-10,133.686
